In [1]:
!pip install langchain-groq==0.1.3 langchain-pinecone chromadb langchain==0.1.17 langchain_community qdrant-client==1.9.1  fastembed==0.2.7 rapidocr-onnxruntime  unstructured[pdf] langchain-groq  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
!pip show langchain_community

Name: langchain-community
Version: 0.0.38
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dataclasses-json, langchain-core, langsmith, numpy, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain


In [3]:
!pip install pinecone-client

In [4]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.1 MB/s eta 0:00:00a 0:00:01


In [5]:
import os
from kaggle_secrets import UserSecretsClient
from pinecone import Pinecone
user_secrets = UserSecretsClient()
os.environ["GROQ_API_KEY"] =  "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"
os.environ["PINECONE_API_KEY"] = "pcsk_5WCgdB_QmWFymLpXWR9CFEaXsPggzWS83xBiBiLmDvWX8RMPdcT6G2x6Wa2p9LEQCvXuC9"

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import FastEmbedEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores.utils import filter_complex_metadata
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_groq import ChatGroq
from langchain.memory import ConversationSummaryMemory
from langchain_pinecone import Pinecone
import uuid

In [7]:
from langchain.memory import ConversationSummaryBufferMemory

In [8]:
from langchain.schema import Document
from docx import Document as DocxDocument

In [9]:
import spacy
import re

In [10]:
class FileProcessor:
    def __init__(self, folder_path, index_name="new", embedding_model="BAAI/bge-base-en-v1.5", chunk_size=1000, chunk_overlap=100):
        self.folder_path = folder_path
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.embeddings = FastEmbedEmbeddings(model_name=embedding_model)
        self.vector_db = Pinecone.from_existing_index(
            index_name=index_name,
            embedding=self.embeddings,
            text_key="text"
        )
        self.nlp = spacy.load("en_core_web_sm")

    def process_files(self):
        """Processes all supported files in the folder."""
        for file_name in os.listdir(self.folder_path):
            file_path = os.path.join(self.folder_path, file_name)
            if file_name.endswith('.pdf') or file_name.endswith('.docx'):
                self.prepare_file(file_path)

    def prepare_file(self, file_path):
        """Reads, splits, and stores embeddings for a single file."""
        pages = self.read_file(file_path)
        texts = self.split_document(pages)
        self.store_embeddings(file_path, texts)

    def read_file(self, file_path):
        """Reads the content of a file based on its extension."""
        if file_path.endswith('.pdf'):
            loader = UnstructuredFileLoader(file_path)
            return loader.load()
        elif file_path.endswith('.docx'):
            return self.read_docx(file_path)
        else:
            raise ValueError("Unsupported file type.")

    def read_docx(self, file_path):
        """Reads a DOCX file and returns Document objects."""
        doc = DocxDocument(file_path)
        return [Document(page_content=p.text) for p in doc.paragraphs if p.text.strip()]

    def split_document(self, pages):
        """Splits the document into smaller chunks."""
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap
        )
        return text_splitter.split_documents(pages)

    def extract_name(self, text):
        """Extracts the name of the person from the CV."""
        doc = self.nlp(text)
        for ent in doc.ents:
            if ent.label_ == "PERSON":
                return ent.text
        return "Unknown Name"

    def store_embeddings(self, file_path, texts):
        """Generates embeddings and stores them with file metadata."""
        # Combine all chunk content into a single text for name extraction
        full_text = " ".join([doc.page_content for doc in texts])
        extracted_name = self.extract_name(full_text)

        # Create metadata and text with embedded name
        metadatas = [
            {
                "source": file_path,
                "id": str(uuid.uuid4()),
                "person_name": extracted_name
            }
            for _ in texts
        ]
        text_contents = [
            f"### CV Analysis for {extracted_name} ###\n\n{doc.page_content}" for doc in texts
        ]
        # Add texts and metadata to vector store
        self.vector_db.add_texts(texts=text_contents, metadatas=metadatas)

In [11]:
class RagChain:
    def __init__(self, folder_path, index_name="new", embedding_model="BAAI/bge-base-en-v1.5", model_name="llama-3.3-70b-versatile", max_token_limit=1024):
        self.folder_path = folder_path
        self.model_name = model_name
        self.groq_client = ChatGroq(temperature=0, model_name=model_name)
        self.embeddings = FastEmbedEmbeddings(model_name=embedding_model)

        # Initialize memory with summary buffer
        self.memory = ConversationSummaryBufferMemory(
            llm=self.groq_client,
            memory_key="chat_history",
            input_key="question",
            max_token_limit=max_token_limit,  # Controls the size of the buffer
            return_messages=True
        )

        # Initialize Pinecone vector store
        self.vector_db = Pinecone.from_existing_index(
            index_name=index_name,
            embedding=self.embeddings,
            text_key="text"
        )
        self.conversational_chain = self.create_qa_chain()

        # Track suggested candidates dynamically
        self.suggested_candidates = set()

    def create_qa_chain(self):
        """Creates a RetrievalQA chain with enhanced prompting and memory."""
        system_template = """
        You are an AI HR assistant. Your goal is to help users identify the best candidates for specific roles based on their requirements.
        Follow these rules:

        1. Use the given context about candidates to identify those who match the user's requirements (e.g., skills, experience).
        2. Rank candidates based on how well they meet the given criteria, explaining your reasoning.
        3. If the user asks for an alternative candidate, avoid suggesting candidates you have already mentioned.
        4. If no matching candidates are found, suggest the closest matches and explain why they could still be valuable.
        5. If the user's query is unclear or lacks details, ask specific follow-up questions to understand their needs better.
        6. If the user asks for candidates in an unrelated field (e.g., HR when context is technical), suggest switching focus or clarifying their request.
        7. When providing comparisons, highlight the strengths, weaknesses, and unique qualities of each candidate to help the user make informed decisions.
        8. Be concise and actionable, and avoid generic responses like "I don't have enough information." Instead, guide the user toward providing more relevant information.

        Context: {context}
        """
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")  # Key: "question"
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

        return RetrievalQA.from_chain_type(
            llm=self.groq_client,
            retriever=self.vector_db.as_retriever(),
            memory=self.memory,
            chain_type="stuff",
            chain_type_kwargs={"prompt": chat_prompt},
            input_key="question"  # Explicitly set the input key to "question"
        )
    def ask_question(self, question):
        """Queries the chain dynamically, handles insufficient input, and utilizes memory."""
        try:
            # Dynamically exclude previously suggested candidates
            filtered_candidates = [{"not": {"person_name": name}} for name in self.suggested_candidates]
            retriever = self.vector_db.as_retriever(
                search_kwargs={"filter": {"$and": filtered_candidates}} if filtered_candidates else {}
            )

            self.conversational_chain.retriever = retriever

            # Pass the question using the correct key "question"
            response = self.conversational_chain.invoke({"question": question})

            # Handle insufficient or unclear responses
            result = response.get("result", "No result found.")
            if "I don't have enough information" in result or "No result found" in result:
                return (
                    "I couldn't find a perfect match for your query. Could you clarify or provide more specific details about "
                    "the skills, experience, or role you're looking for?"
                )

            # Extract candidate name dynamically from the response
            suggested_candidate = self.extract_candidate_from_response(result)
            if suggested_candidate:
                self.suggested_candidates.add(suggested_candidate)

            return result
        except Exception as e:
            print(f"Error while querying the chain: {e}")
            return "An error occurred while processing your question."

    def extract_candidate_from_response(self, response):
        """Extracts the suggested candidate's name from the response."""
        match = re.search(r"Candidate Name: ([\w\s]+)", response)
        if match:
            return match.group(1)
        return None

    def display_memory(self):
        """Displays the summarized memory."""
        try:
            memory_summary = self.memory.load_memory_variables({}).get("chat_history", "")
            if memory_summary:
                print("Memory Summary:")
                print(memory_summary)
            else:
                print("No memory has been summarized yet.")
        except Exception as e:
            print(f"Error while retrieving memory: {e}")

In [12]:
folder_path = "/kaggle/input/cvs-folder"

In [13]:
processor = FileProcessor(folder_path)
processor.process_files()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [14]:
rag_chain = RagChain(folder_path)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
response = rag_chain.ask_question("Who is the best candidate for data testing, Kafka, and NiFi?")
print(response)

In [ ]:
rag_chain.display_memory()

In [ ]:
response = rag_chain.ask_question("could you tell all skills reem has?")
print(response)

In [15]:
response = rag_chain.ask_question("Who is the best 5 candidates for data testing?")
print(response)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Based on the provided context, I can only identify one candidate, Reem Ayman, who has experience in data testing. Here's a brief analysis of her qualifications:

1. Reem Ayman: As a Data/ML Testing Engineer, Reem has experience in data testing, having collaborated with the data testing team to implement a Python package for automated analysis of a supervised machine learning model. She also has experience in software testing, including manual, automation, API, and database testing.

Since there are no other candidates mentioned in the context, I'll need more information to provide a list of the top 5 candidates for data testing. Could you please provide more context or details about the other candidates, such as their CVs or experience in data testing? Alternatively, if you'd like to expand the search, I can ask some follow-up questions to help identify potential candidates:

* What specific skills or qualifications are you looking for in a data testing candidate?
* Are there any speci

In [16]:
response = rag_chain.ask_question("Who else?")
print(response)

It seems we have a limited pool of candidates so far. To provide more suggestions, I'd like to know more about the role you're trying to fill. Can you please provide me with some details about the position, such as:

* What type of industry or field is the role in?
* What are the key skills and qualifications required for the position?
* Are there any specific experiences or certifications you're looking for in a candidate?

This will help me provide more tailored suggestions and potentially introduce new candidates that might be a good fit for the role.


In [ ]:
response = rag_chain.ask_question("Who are the best 5 candidates for data science?")
print(response)

In [ ]:
response = rag_chain.ask_question("could you did this evaluation for me?")
print(response)

In [ ]:
response = rag_chain.ask_question("could you list all candidates provided in general and their scope?")
print(response)

In [ ]:
response = rag_chain.ask_question("who else?")
print(response)

In [1]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.2
aiofiles==22.1.0
aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
aiosqlite==0.20.0
alabaster==0.7.16
albucore==0.0.16
albumentations==1.4.15
alembic==1.14.0
altair==5.5.0
annotated-types==0.7.0
annoy==1.17.3
ansicolors==1.1.8
antlr4-python3-runtime==4.9.3
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
args==0.1.0
array_record==0.5.1
arrow==1.3.0
arviz==0.19.0
astropy==6.1.3
astropy-iers-data==0.2024.9.16.0.32.21
asttokens==3.0.0
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==24.2.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
bayesian-optimization==2.0.1
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.17.0
bigquery-magics==0.2.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blobfile==3.0.0
blosc2==2.0.0
bokeh==3.4.3
Boruta==0.4.3
boto3==1.35.83
botocore==1.35.83
-e git+https://github.com/SohierDane/BigQuery_Helper@8615a7f6c1663e7f2d48aa2b32c2dbcb600a440f#egg=bq_helper
bqplot==0.12.43
branca==0.7.2
b

In [17]:
!python --version
!pip --version

Python 3.10.12


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [18]:
!pip freeze > requirements.txt